In [54]:
import jinja2 as jin
import pandas as pd

In [55]:
template = jin.Template("""
<!DOCTYPE html>
  <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>RMHPA</title>
      <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bulma/0.7.2/css/bulma.min.css">
      <style>
        body{
          width: 900px;
          margin-left: auto;
          margin-right: auto;
        }
      </style>

  </head>
  <body>
    <h1 class="title">RMHPA</h1>

<div class="tabs">
  <ul>
    <li ><a href="index.html">Pilots</a></li>
    <li class="is-active"><a href="sorted.html">Flights</a></li>
  </ul>
</div>


    <table class="table">
     <tr>
         <th> index </th>
         <th> date </th>
         <th> name </th>
         <th> distance </th>
         <th> XC km</th>
         <th> IGC file </th>
     </tr>
     {% for row in rows %}
     <tr>
         <td> {{row['index']}}</td>
         <td> {{row['date']}}</td>
         <td> <a href="{{row['name_url']}}.html"> {{row['name']}}</a> </td>
         <td> {{row['distance']}}</td>
         <td> {{row['xckm']}}</td>
         <td> <a href="flights/flight_{{row['flight_number']}}.igc" >flight_{{row['flight_number']}}.igc</a></td>
     </tr>
    {% endfor %}
    
    </table>
  </body>

</html>
""")

name_template = jin.Template("""
<!DOCTYPE html>
  <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>RMHPA</title>
      <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bulma/0.7.2/css/bulma.min.css">
      <style>
        body{
          width: 900px;
          margin-left: auto;
          margin-right: auto;
        }
      </style>

  </head>
  <body>
    <h1 class="title">RMHPA</h1>
    
<div class="tabs">
  <ul>
    <li ><a href="index.html">Pilots</a></li>
    <li ><a href="sorted.html">Flights</a></li>
  </ul>
</div>

    <table class="table">
     <tr>
         <th> index </th>
         <th> date </th>
         <th> name </th>
         <th> distance </th>
         <th> XC km</th>
         <th> IGC file </th>
     </tr>
     {% for row in rows %}
     <tr>
         <td> {{row['index']}}</td>
         <td> {{row['date']}}</td>
         <td> <a href="{{row['name_url']}}.html"> {{row['name']}}</a></td>
         <td> {{row['distance']}}</td>
         <td> {{row['xckm']}}</td>
         <td> <a href="flights/flight_{{row['flight_number']}}.igc" >flight_{{row['flight_number']}}.igc</a></td>
     </tr>
    {% endfor %}
    
    </table>
  </body>

</html>
""")

In [56]:
df = pd.read_csv("clean/rmhpa_flights.csv")
rows = df.to_dict(orient="records")

for row in rows:
    row['flight_number'] = row['links'].split('/')[-1] 
    row['name_url'] = row['name'].replace(" ", "_").replace("'", "").lower()

with open("../docs/sorted.html", 'w') as outputfile:
    outputfile.write(template.render({'rows': rows}))
    


In [57]:
names = []

for group in df.groupby('name'):
    name = group[0]
    rows = group[1].to_dict(orient="records")
    name_url = name.replace(" ", "_").replace("'", "").lower()
    
    tmp = {}
    tmp['name'] = name
    tmp['name_url'] = name_url
    names.append(tmp)
    
    for row in rows:
        row['flight_number'] = row['links'].split('/')[-1] 
        
    with open("../docs/{}.html".format(name_url), 'w') as outputfile:
        outputfile.write(name_template.render({'rows': rows,
                                               'name': name}))

In [58]:
names = sorted(names, key=lambda x: x['name'])

In [60]:
pilot_template = jin.Template("""
<!DOCTYPE html>
  <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>RMHPA</title>
      <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bulma/0.7.2/css/bulma.min.css">
      <style>
        body{
          width: 900px;
          margin-left: auto;
          margin-right: auto;
        }
        .mybox{
            width: 200px;
            float: left;
        }
      </style>

  </head>
  <body>
    <h1 class="title">RMHPA</h1>

<div class="tabs">
  <ul>
    <li class="is-active"><a href="index.html">Pilots</a></li>
    <li><a href="sorted.html">Flights</a></li>
  </ul>
</div>


    <div>
     
         {% for name in names %}
         <div class="mybox">
            <a href="{{name['name_url']}}.html"> {{name['name']}}</a> 
         </div>
        {% endfor %}
    
    </div>
    
    
  </body>

</html>
""")

with open("../docs/index.html", 'w') as outfile:
    outfile.write(pilot_template.render({ 'names': names }))